In [1]:
import os
import numpy as np
import tensorflow as tf
import time

import matplotlib.pyplot as plt

import keras
from keras.layers import Activation, Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
import pandas as pd




/home/sauban/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sauban/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sauban/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sauban/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
def deep_cnn_model(input_shape):
    model = Sequential()

    model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(64,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [22]:
data_f = "./data/"
train = np.genfromtxt(os.path.join(data_f,"train.csv"), delimiter=',')
test = np.genfromtxt(os.path.join(data_f,"test.csv"), delimiter=',')


In [4]:
# val = train[1:2000,:]
# train = train[2000:,:]

In [6]:
x_train = train[1:,1:]
y_train = train[1:,0]


# x_val = val[:,1:]
# y_val = val[:,0]

In [7]:
x_train = x_train.reshape((-1,28,28,1))
# x_val = x_val.reshape((-1,28,28,1))


In [8]:
# x_train= tf.expand_dims(x_train, axis=3)
# x_val= tf.expand_dims(x_val, axis=3)

In [9]:
cnn_model = deep_cnn_model((28,28,1))
cnn_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 32)        25632     
_________________________________________________________________
batch_normalization_3 (Batch (None, 12, 12, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)       

In [10]:
to_categorical(y_train)[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [15]:
cnn_model.fit(x_train, to_categorical(y_train), epochs=5, batch_size=128)

Epoch 1/5
42000/42000 [==============================] - 47s 1ms/step - loss: 0.0363 - accuracy: 0.9886
Epoch 2/5
42000/42000 [==============================] - 46s 1ms/step - loss: 0.0322 - accuracy: 0.9898
Epoch 3/5
42000/42000 [==============================] - 46s 1ms/step - loss: 0.0300 - accuracy: 0.9909
Epoch 4/5
42000/42000 [==============================] - 45s 1ms/step - loss: 0.0282 - accuracy: 0.9910
Epoch 5/5
42000/42000 [==============================] - 47s 1ms/step - loss: 0.0237 - accuracy: 0.9925


In [16]:
# evaluate the keras model
_, accuracy = cnn_model.evaluate(x_train, to_categorical(y_train))
print('Train Accuracy: %.2f' % (accuracy*100))
# _, accuracy = cnn_model.evaluate(x_val, to_categorical(y_val))
# print('Test Accuracy: %.2f' % (accuracy*100))



42000/42000 [==============================] - 9s 223us/step
Train Accuracy: 99.63


In [23]:
test = test[1:,:]
test = test.reshape((-1,28,28,1))
preds_test = np.argmax(cnn_model.predict(test),axis=1)

In [25]:
# Save test predictions to file
output = pd.DataFrame({'ImageId': list(range(1,test.shape[0]+1)),
                       'Label': preds_test})
output.to_csv('submission.csv', index=False)

In [24]:
len(preds_test)

28000